# ant

> Meta Tools for AI agents

In [ ]:
#| default_exp core

In [ ]:
#| export
import requests

### Getting L402 URI Info

In [ ]:
#| export
def get_l402_uri_info(uri: str) -> dict:
    """
   Returns the information of the L402 URI resource.

    Args:
    uri (str): The L402 URI to describe.

    Returns:
    dict: A dictionary containing the description of the L402 URI.
    """
    if not uri.startswith("l402://"):
        raise ValueError("Invalid L402 URI format")
    
    scheme = "http://" if "localhost" in uri else "https://"
    http_url = uri.replace("l402://", scheme, 1)
    
    response = requests.get(http_url)
    response.raise_for_status()
    
    return response.json()

    

In [ ]:
uri = "l402://api.fewsats.com/v0/gateway/f12e5deb-b07b-4af4-a4f2-3fbf076228a9/info" # web scraping endpoint

info = get_l402_uri_info(uri)
info['access'].pop('authentication', None)
info

{'access': {'endpoint': 'https://api.fewsats.com/v0/gateway/access/f12e5deb-b07b-4af4-a4f2-3fbf076228a9',
  'method': 'POST'},
 'content_type': 'api',
 'cover_url': '',
 'description': 'Scrape a given URL. Pass the URL as JSON in the request body as follows:\n```\n {"url": url }\n```',
 'name': 'Web Scraper TF',
 'pricing': [{'amount': 1, 'currency': 'USD'}],
 'version': '0.1'}

### Generating Python Function

In [ ]:
#| export

from cosette import Chat
import json 

func_generation_sp = """You are an AI assistant specialized in creating Python functions based on L402 info inputs. 
When given an L402 info dictionary, your task is to generate a Python function that can access the specified endpoint. 
Follow these guidelines:

1. Create a function name that relates to the resource or action described in the 'name' field making it specific enough to avoid conflicts.
2. Use the 'endpoint' field to determine the URL for the request.
3. Use the 'method' field to determine the HTTP method for the request.
4. Handle any required parameters for the endpoint request by passing them as function params.
5. Write a docstring that includes:
   - A brief description of the function's purpose
   - Parameters with their types and descriptions
   - Return value with its type and description
6. Do not import the requests library. Assume it is available in the global scope.
7. Handle potential errors and exceptions appropriately.
8. Do not handle any authentication or authorization in the function.
9. Return the response from the endpoint.
10. Use L402 request client to send the request and configure exactly like this:

L402 Request configuration:
```
l402_requests.configure(
   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),
   credentials_service=SqliteCredentialsService()
)
```

Example input:
```
{'access': {'authentication': {'format': 'L402 {credentials}:{proof_of_payment}',
   'header': 'Authorization',
   'protocol': 'L402'},
  'endpoint': 'https://blockbuster.fewsats.com/video/stream/79c816f77fdc4e66b8cd18ad67537936',
  'method': 'POST'},
 'content_type': 'video',
 'cover_url': 'https://pub-3c55410f5c574362bbaa52948499969e.r2.dev/cover-images/79c816f77fdc4e66b8cd18ad67537936',
 'description': 'Lex Fridman Podcast full episode:    • Andrew Huberman: Focus, Controversy, ',
 'name': 'How to focus and think deeply | Andrew Huberman and Lex Fridman',
 'pricing': [{'amount': 1, 'currency': 'USD'}],
 'version': '1.0'}
```

Example output:
```
from l402.client import requests as l402_requests
from l402.client.preimage_provider import AlbyAPI
from l402.client.credentials import SqliteCredentialsService
import requests
import os

l402_requests.configure(
   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),
   credentials_service=SqliteCredentialsService()
)
 def stream_video_huberman() -> dict:
    \"\"\"
    Stream the video of the podcast episode featuring Andrew Huberman and Lex Fridman.

    This function sends a POST request to the specified endpoint.
    Returns:
    - requests.Response: The response object containing the server's response to the request.
    \"\"\"
    endpoint = "https://blockbuster.fewsats.com/video/stream/79c816f77fdc4e66b8cd18ad67537936"
    try:
        response = l402_requests.post(endpoint, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None
```
Your response should be a complete Python function definition, including imports, type hints, docstring, and function body. Only output the function definition, no other text."""


In [ ]:
#| export

def generate_python_function(info: dict) -> str:
   """
   Generates a Python function based on the provided L402 info dictionary.

   Args:
   info (dict): The L402 info dictionary containing the details of the resource.

   Returns:
   str: A Python function definition as a string.
   """
   # we remove authentication because it's handled by the L402 client
   # and it only confuses the LLM
   info['access'].pop('authentication', None)

   chat = Chat('gpt-4o-mini', sp=func_generation_sp)
   return chat(json.dumps(info))


In [ ]:

func_code = generate_python_function(info)

### Extracting the function code

In [ ]:
#| export
def get_text(response):
    return response.choices[0].message.content


In [ ]:

response = get_text(func_code)
response

'```python\nfrom l402.client import requests as l402_requests\nfrom l402.client.preimage_provider import AlbyAPI\nfrom l402.client.credentials import SqliteCredentialsService\nimport requests\nimport os\n\nl402_requests.configure(\n   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),\n   credentials_service=SqliteCredentialsService()\n)\n\ndef post_web_scraper_access(url: str) -> dict:\n    """\n    Scrape a given URL by passing it as JSON in the request body.\n\n    This function sends a POST request to the specified endpoint with the URL in the request body.\n    \n    Parameters:\n    - url (str): The URL to be scraped.\n\n    Returns:\n    - dict: The response object containing the server\'s response to the request.\n    """\n    endpoint = "https://api.fewsats.com/v0/gateway/access/f12e5deb-b07b-4af4-a4f2-3fbf076228a9"\n    body = {"url": url}\n    \n    try:\n        response = l402_requests.post(endpoint, json=body)\n        response.raise_for_status()  # Raise an erro

In [ ]:
#| export

import re
def extract_function_code(func_code):
    code_pattern = re.compile(r'```python\n(.*?)```', re.DOTALL)
    match = code_pattern.search(func_code)
    return match.group(1).strip() if match else func_code


In [ ]:
extracted_code = extract_function_code(response)
print(extracted_code)

from l402.client import requests as l402_requests
from l402.client.preimage_provider import AlbyAPI
from l402.client.credentials import SqliteCredentialsService
import requests
import os

l402_requests.configure(
   preimage_provider=AlbyAPI(api_key=os.getenv("ALBY_TOKEN")),
   credentials_service=SqliteCredentialsService()
)

def post_web_scraper_access(url: str) -> dict:
    """
    Scrape a given URL by passing it as JSON in the request body.

    This function sends a POST request to the specified endpoint with the URL in the request body.
    
    Parameters:
    - url (str): The URL to be scraped.

    Returns:
    - dict: The response object containing the server's response to the request.
    """
    endpoint = "https://api.fewsats.com/v0/gateway/access/f12e5deb-b07b-4af4-a4f2-3fbf076228a9"
    body = {"url": url}
    
    try:
        response = l402_requests.post(endpoint, json=body)
        response.raise_for_status()  # Raise an error for bad responses
        return respon

### Creating the function

In [ ]:
#| export

import importlib.util
import re
import os

def create_func(code_string):
    # Ensure the funcs directory exists
    os.makedirs('.funcs', exist_ok=True)

    # Extract Python code from Markdown code blocks if present
    code_pattern = re.compile(r'```python\n(.*?)```', re.DOTALL)
    match = code_pattern.search(code_string)
    extracted_code = match.group(1).strip() if match else code_string

    # Create a temporary file name

    function_name = extracted_code.split('def ')[1].split('(')[0].strip()

    temp_file = f'.funcs/{function_name}.py'

    # Write the code to a temporary file
    with open(temp_file, 'w') as f:
        f.write(extracted_code)

    # Import the function
    spec = importlib.util.spec_from_file_location("temp_module", temp_file)
    temp_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(temp_module)

    # Get the function
    function_name = extracted_code.split('def ')[1].split('(')[0].strip()
    return getattr(temp_module, function_name)



In [ ]:

gf = create_func(extracted_code)


In [ ]:
r = gf('https://www.fewsats.com/')
r

{'success': True,
 'data': {'markdown': "![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/66475f66cdbcb00d7bf2fd44_Employee%20data.png)\n\n[Learn More](#)![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)\n\n\n![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)\n\n[Learn More](#)![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)\n\n\n![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)\n\n![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/664773c569f5d9dbac6655a0_Employee%20data-2.png)\n\n![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/664762e683491e5b21c08f31_Employee%20data-3.png)\n\n[Learn More](#)![](https://cdn.prod.website-files.com/6645a4a7082c15bec7e9fd6e/6645a4a7082c15bec7e9fdda_arrow-right-icon.png)\

## Adding Tools to the Agent

In [ ]:
tools = [get_l402_uri_info]
chat = Chat('gpt-4o-mini', sp='You are a helpful assistant that provides information about L402 URIs and your available tools.', tools=tools)

chat("which tools do you have available?")

In [ ]:
r = chat(f'What is the info of: {uri}')
display(Markdown(get_text(r)))

In [ ]:
# Step 1: Get L402 URI info
info = get_l402_uri_info(uri)
print("L402 URI info retrieved successfully.")
info


In [ ]:
# Step 2: Generate Python function
func_code = generate_python_function(info)
print("Python function generated successfully.")
func_code

In [ ]:

# Step 3: Extract function code
extracted_code = extract_function_code(get_text(func_code))
print("Function code extracted successfully.")
extracted_code

In [ ]:

# Step 4: Create the function
cf = create_func(extracted_code)
print("Function created successfully.")

In [ ]:
tools = [get_l402_uri_info, cf]

In [ ]:
chat = Chat('gpt-4o-mini', sp='You are a helpful assistant that provides information about L402 URIs and your available tools.', tools=tools)


In [ ]:
chat("which tools do you have available?")

In [ ]:
chat.toolloop("can you scrape this url for me: https://www.fewsats.com/")

Now that we have the full process on how to add a tool to the agent, let's write a single function that encapsulates the entire process. The agent will be able to call this function to add a new tool.

In [ ]:
#| export

def add_l402_tool(uri: str) -> str:
    """Add a new tool to the agent's toolset."""
    info = get_l402_uri_info(uri)
    r = generate_python_function(info)
    func_code = extract_function_code(get_text(r))

    cf = create_func(func_code)
    tools.append(cf)
    return f"tool {cf.__name__} added"

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()